<a href="https://colab.research.google.com/github/luqmansen/notebook-collection/blob/master/kNN_fruits_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Mata Kuliah Data Mining**
## Klasifikasi Data Citra menggunakan KNN

1. Luqman Setyo / Nugroho 21120117120008

2. Heidy novendra / 21120117130035

3. Faiz Noerdiyan / 21120117130060

4. Chandra Purnama / 21120117130070

Sumber dataset

Fruits 360

A dataset with 82213 images of 120 fruits and vegetables

https://www.kaggle.com/moltean/fruits

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/knn-fruits-classifier'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/knn-fruits-classifier


In [0]:
!wget 'https://storage.googleapis.com/kaggle-data-sets/5857/778601/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705118&Signature=ixhgjk387ewcAIc5nxNyA0fzhkf1yp1d89dkJjr820w05jsfnmB%2BndEuUgHLjg9KpnSCTaFIvYO6VkWIIPNiMRAj5uXuPykG8q%2FS4ddd696Q8V2qPwqXQDxFDFCpb6NnOn%2FuWJ9icVbxvHUUKAeZkjuoFaG%2FVipVbbEGvqG5TiINYxu4%2FkN4S3YPs%2FBZmlUpGHYPyfpIROxeb7JVhZSsjrhTJcGmuG4kftnMldNGvWl8qQMXFjVe2EMpTLi40HTu7L1gHvDIOIx4numHlZOQV8qWztMcKAXahH5RKDkeZD9z%2Fm37WLwcXZ%2FP7pgO8xglGDhX4X4AHtKjAcMsNL9GAQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dfruits.zip'

The name is too long, 522 chars total.
Trying to shorten...
New name is archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705118&Signature=ixhgjk387ewcAIc5nxNyA0fzhkf1yp1d89dkJjr820w05jsfnmB+ndEuUgHLjg9KpnSCTaFIvYO6VkWIIPNiMRAj5uXuPykG8q%2FS4ddd696Q8V2qPwqXQDxFDFCpb6NnOn%.
--2020-04-09 15:27:47--  https://storage.googleapis.com/kaggle-data-sets/5857/778601/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705118&Signature=ixhgjk387ewcAIc5nxNyA0fzhkf1yp1d89dkJjr820w05jsfnmB%2BndEuUgHLjg9KpnSCTaFIvYO6VkWIIPNiMRAj5uXuPykG8q%2FS4ddd696Q8V2qPwqXQDxFDFCpb6NnOn%2FuWJ9icVbxvHUUKAeZkjuoFaG%2FVipVbbEGvqG5TiINYxu4%2FkN4S3YPs%2FBZmlUpGHYPyfpIROxeb7JVhZSsjrhTJcGmuG4kftnMldNGvWl8qQMXFjVe2EMpTLi40HTu7L1gHvDIOIx4numHlZOQV8qWztMcKAXahH5RKDkeZD9z%2Fm37WLwcXZ%2FP7pgO8xglGDhX4X4AHtKjAcMsNL9GAQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dfruits.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.

In [0]:
!unzip -q archive*

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import time

In [0]:
def image_to_feature_vector(image, size=(32, 32)):
	# Konversi gambar menjadi berukuran 32x32 
    
  # return berupa list of number 32x32x3 (3 = channel rgb) = list of 3.072 
	return cv2.resize(image, size).flatten()

In [0]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# konversi gambar ke hsv
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # menghitung histogram gambar
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
	
  # Normalisasi histogram
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	else:
		cv2.normalize(hist, hist)
  
	# return berupa feature vector 512-d (8 untuk tiap H,S,V sehingga 8x8x8)
	return hist.flatten()

In [0]:
imagePaths = list(paths.list_images("fruits-360_dataset/fruits-360/Training"))
# initialize the raw pixel intensities matrix, the features matrix, and labels list
rawImages = []
features = []
labels = []

In [0]:
test_imagePaths = list(paths.list_images("fruits-360_dataset/fruits-360/Test"))
# initialize the raw pixel intensities matrix, the features matrix, and labels list
test_rawImages = []
test_features = []
test_labels = []

In [0]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
  # load the image and extract the class label (assuming that our
  # path as the format: /path/to/dataset/{class}.{image_num}.jpg
  image = cv2.imread(imagePath)
  label = imagePath.split("/")[-2]
  # extract raw pixel intensity "features", followed by a color
  # histogram to characterize the color distribution of the pixels in the image
  pixels = image_to_feature_vector(image)
  hist = extract_color_histogram(image)
  # update the raw images, features, and labels matricies, respectively
  rawImages.append(pixels)
  features.append(hist)
  labels.append(label)
	# show an update every 1,000 images
  if i > 0 and i % 1000 == 0:
    print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 1000/60498
[INFO] processed 2000/60498
[INFO] processed 3000/60498
[INFO] processed 4000/60498
[INFO] processed 5000/60498
[INFO] processed 6000/60498
[INFO] processed 7000/60498
[INFO] processed 8000/60498
[INFO] processed 9000/60498
[INFO] processed 10000/60498
[INFO] processed 11000/60498
[INFO] processed 12000/60498
[INFO] processed 13000/60498
[INFO] processed 14000/60498
[INFO] processed 15000/60498
[INFO] processed 16000/60498
[INFO] processed 17000/60498
[INFO] processed 18000/60498
[INFO] processed 19000/60498
[INFO] processed 20000/60498
[INFO] processed 21000/60498
[INFO] processed 22000/60498
[INFO] processed 23000/60498
[INFO] processed 24000/60498
[INFO] processed 25000/60498
[INFO] processed 26000/60498
[INFO] processed 27000/60498
[INFO] processed 28000/60498
[INFO] processed 29000/60498
[INFO] processed 30000/60498
[INFO] processed 31000/60498
[INFO] processed 32000/60498
[INFO] processed 33000/60498
[INFO] processed 34000/60498
[INFO] processed 35000/

In [0]:
# loop over the input images
for (i, imagePath) in enumerate(test_imagePaths):
  # load the image and extract the class label (assuming that our
  # path as the format: /path/to/dataset/{class}.{image_num}.jpg
  image = cv2.imread(imagePath)
  label = imagePath.split("/")[-2]
  # extract raw pixel intensity "features", followed by a color
  # histogram to characterize the color distribution of the pixels in the image
  pixels = image_to_feature_vector(image)
  hist = extract_color_histogram(image)
  # update the raw images, features, and labels matricies, respectively
  test_rawImages.append(pixels)
  test_features.append(hist)
  test_labels.append(label)
	# show an update every 1,000 images
  if i > 0 and i % 1000 == 0:
    print("[INFO] processed {}/{}".format(i, len(test_imagePaths)))

[INFO] processed 1000/20622
[INFO] processed 2000/20622
[INFO] processed 3000/20622
[INFO] processed 4000/20622
[INFO] processed 5000/20622
[INFO] processed 6000/20622
[INFO] processed 7000/20622
[INFO] processed 8000/20622
[INFO] processed 9000/20622
[INFO] processed 10000/20622
[INFO] processed 11000/20622
[INFO] processed 12000/20622
[INFO] processed 13000/20622
[INFO] processed 14000/20622
[INFO] processed 15000/20622
[INFO] processed 16000/20622
[INFO] processed 17000/20622
[INFO] processed 18000/20622
[INFO] processed 19000/20622
[INFO] processed 20000/20622


In [0]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
test_rawImages = np.array(test_rawImages)
test_features = np.array(test_features)
test_labels = np.array(test_labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))
print("[INFO] test pixels matrix: {:.2f}MB".format(test_rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] test features matrix: {:.2f}MB".format(test_features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 181.49MB
[INFO] features matrix: 121.00MB
[INFO] test pixels matrix: 61.87MB
[INFO] test features matrix: 41.24MB


In [0]:
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(rawImages, labels)
acc = model.score(test_rawImages, test_labels)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 93.56%


In [0]:
# train and evaluate a k-NN classifer on the histogram
print("[INFO] evaluating feature pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(features, labels)
acc = model.score(test_features, test_labels)
print("[INFO] histogram pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] histogram pixel accuracy: 98.00%


## Hyperparameter tunning

n_neghbors : jumlah NN k (dari 0 hingga 29, exclusive)

metrics : distance function metrics

In [0]:
params = {"n_neighbors": np.arange(1, 31, 2),
	"metric": ["euclidean", "cityblock"]}

In [0]:
model = KNeighborsClassifier(n_jobs=-1)

In [0]:
# Grid Search tuning
print("[INFO] tuning hyperparameters via grid search")
grid = GridSearchCV(model, params, verbose=2)
start = time.time()
grid.fit(features, labels)
# evaluate the best grid searched model on the testing data
print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(test_features, test_labels)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [0]:
# tune the hyperparameters via a randomized search
print("[INFO] tuning hyperparameters via randomized search")
grid = RandomizedSearchCV(model, params)
start = time.time()
grid.fit(features, labels)
# evaluate the best randomized searched model on the testing data
print("[INFO] randomized search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(test_features, test_labels)
print("[INFO] randomized search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via randomized search
[INFO] randomized search took 3870.37 seconds
[INFO] randomized search accuracy: 99.22%
[INFO] randomized search best parameters: {'n_neighbors': 1, 'metric': 'cityblock'}
